In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#| code-fold: false
# IMPORTING LIBRARIES
# --------------------

# AstroML
from astroML.datasets import fetch_LINEAR_sample
from astropy.timeseries import LombScargle
from astroML.datasets import fetch_LINEAR_sample
from astroML.datasets import fetch_LINEAR_geneva
from astropy.timeseries import TimeSeries
from astropy.table import Table
from astroML.time_series import MultiTermFit

# ZTF
from ztfquery import lightcurve

# Basic libraries
import pickle
import os
import sys
from tqdm import tqdm

# Plotting
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib import ticker
import matplotlib.colors as mcolors
from matplotlib.font_manager import FontProperties

# DataFrame analysis
import pandas as pd

# Math libraries
import numpy as np
import scipy as sc
from scipy.stats import norm

# CONFIGURATION
# -------------
sns.set_theme(style='white') # setting the theme for plotting
sys.path.insert(0,'../src/')
np.random.seed(42)

# configuring plotting colors
colors = ['#465BBB', '#3F8FCE', '#7ABBCE', '#3A3865', '#A82F43', '#612A37', '#DC5433', '#F29457']
b1 = '#465BBB'
b2 = '#3F8FCE'
b3 = '#7ABBCE'
b4 = '#3A3865'
black1 = '#22212A'
black2 = '#2D1E21'
or1 = '#A82F43'
or2 = '#612A37'
or3 = '#DC5433'
or4 = '#F29457'
muted_colors = sns.set_palette(sns.color_palette(colors))

# configuring fonts for plotting
font = FontProperties()
font.set_family('serif')
font.set_name('Andale Mono')
font.set_style('normal')
#font.set_size('normal')


%matplotlib inline 

In [3]:
#| code-fold: false
# Importing custom libraries
# ----------------------------
from data_access import*
from helper import*
from RR_lyrae_params import*
from blazhko_analysis import*
from BE_plotting import*

# Visual analysis

The final step of this project is to visually analyze and confirm the algorithms selection of Blazhko candidates. We perform this operation using a custom made interface that displays 4 graphs, letting the user analyze them and click 'KEEP' if it confirms the star is Blazhko, or 'CONTINUE' if it is not.

In [4]:
# DATA
# ======
# LINEAR data
dataL = fetch_LINEAR_sample(data_home='../inputs')

# calcualted data
end = 'ffinal'

# light curve table
lc_analysis = pd.read_csv("../outputs/LC_ANALYSIS_"+end+".csv")
# rr lyrae
rrlyrae = pd.read_csv('../outputs/Lrrlyr_unprocessed.csv')
# blazhko candidates
blazhko_candidates = pd.read_csv('../outputs/blazhko_canmodfinal.csv')
# periodogram data
periodogr = pickle.load(open("../outputs/periodograms_"+end+".pkl", "rb"))
# fits data
fits = np.load("../outputs/fits_"+end+".npy", allow_pickle=True)
# ztf data
ztf_data = np.load("../outputs/ztf_data_filter"+end+".npy", allow_pickle=True)

This process was conducted in 4 phases:
1. ### The phased graph
Firstly, the shape and noisiness of the phased light curves were examined. If the light curve had the correct shape, there wasn't too much noise ($\chi^2$ wasn't too high) and there appeared to be smearing of the phased light curve ( a sign of modulation ), the light curve passed the first phase.

2. ### The periodogram
Secondly, the correctness of the algorithm in recognizing the blazhko frequency was looked at. If the blazhko peaks are right next to the yearly alias and not statistically significant, or if the algorithm detected a false signal, the light curve wouldn't pass this phase.

3. ### The full data
Thirdly, the general shape of the light curve was looked at. If the data showed a wave-like pattern, it would indicate the presence of amplitude modulation.

4. ### Seasons of observation
Finally, the most important phase, where we analyzed the light curve fit for each observation season, comparing how the light curve changed depending on the phased and normalized graph. We look for a translation in the x-coordinate, or a *phase modulation*, or if the width of the light curve changes during seasons, we found *amplitude modulation*.

A star can satisfy all 4 criteria, the first and final stage, or just the second stage to be considered a Blazhko star.

We split up our 409 candidates into 3 categories for more efficient visual analysis:
1. ZTF periodogram group
2. LINEAR periodogram group
3. Other metrics group

## Periodogram group

In [ ]:
# ZTF PERIODOGRAM
# =================
ztf_per = category_analysis(blazhko_candidates, fits, periodogr, ztf_data, dataL,id_list=None,parameter='IndicatorType', value='Z')

In [ ]:
blazhko_analyzer = ztf_per.get_save_data()
end = 'Z_p'
blazhko_analyzer.to_csv("../outputs/group"+end+".csv", index=False)

print(blazhko_analyzer.shape)

In [ ]:
# LINEAR PERIODOGRAM
# ======================
linear_per = category_analysis(blazhko_candidates, fits, periodogr, ztf_data, dataL,id_list=None,parameter='IndicatorType', value='L')

In [ ]:
blazhko_analyzerl = linear_per.get_save_data()
end = 'L_p'
blazhko_analyzerl.to_csv("../outputs/group"+end+".csv", index=False)

print(blazhko_analyzerl.shape)

## Score group

In [ ]:
# OTHER
# =======
be_other = blazhko_candidates[(blazhko_candidates['IndicatorType']!= 'Z') & (blazhko_candidates['IndicatorType']!= 'L')]
be_other = be_other.reset_index(drop=True)
print(be_other.shape)
be_other.head()

In [ ]:
score = category_analysis(be_other, fits, periodogr, ztf_data, dataL, 'other')

In [ ]:
blazhko_analyzerother = score.get_save_data()
end = 'score'
blazhko_analyzerother.to_csv("../outputs/group"+end+".csv", index=False)

print(blazhko_analyzerother.shape)

---